In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, Flatten
import matplotlib.pyplot as plt


# Load and preprocess the Fashion MNIST dataset
(X_train, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
X_train = (X_train - 127.5) / 127.5  # Normalize to [-1, 1]
X_train = X_train.reshape(X_train.shape[0], -1)  # Flatten images (28x28 -> 784)

# Define constants
noise_dim = 100  # Noise vector size
image_dim = X_train.shape[1]  # 28*28 = 784
batch_size = 128
epochs = 5000

# Build the generator
def build_generator():
    model = Sequential([
        Dense(128, input_dim=noise_dim),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(image_dim, activation='tanh'),
    ])
    return model

# Build the discriminator
def build_discriminator():
    model = Sequential([
        Dense(256, input_dim=image_dim),
        LeakyReLU(alpha=0.2),
        Dense(128),
        LeakyReLU(alpha=0.2),
        Dense(1, activation='sigmoid'),
    ])
    return model

generator = build_generator()
discriminator = build_discriminator()

discriminator.trainable = True  # Enable training
discriminator.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
gan = Sequential([generator, discriminator])
gan.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
    loss='binary_crossentropy'
)

# Function to generate and display images
def generate_images(epoch, generator, examples=10):
    noise = np.random.normal(0, 1, (examples, noise_dim))
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=(10, 1))
    for i in range(examples):
        plt.subplot(1, examples, i + 1)
        plt.imshow(generated_images[i], cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Training loop
real_labels = np.ones((batch_size, 1))
fake_labels = np.zeros((batch_size, 1))

for epoch in range(epochs + 1):
    # Train the discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_images = X_train[idx]

    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    fake_images = generator.predict(noise)

    # Debugging shapes
    print("Epoch:", epoch)
    print("Real images shape:", real_images.shape)
    print("Fake images shape:", fake_images.shape)
    print("Real labels shape:", real_labels.shape)
    print("Fake labels shape:", fake_labels.shape)
    print("Discriminator trainable:", discriminator.trainable)
    print("GAN summary:")
    gan.summary()

    d_loss_real = discriminator.train_on_batch(real_images, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
    discriminator.trainable = False
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    g_loss = gan.train_on_batch(noise, real_labels)

    # Print progress and generate images every 1000 epochs
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc.: {100 * d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")
        generate_images(epoch, generator)
